### A/B testing

In [128]:
import numpy as np
from numpy.random import permutation , choice
np.set_printoptions(precision=4)

In [89]:
groupA = np.array( [1] * 18 + [0] * 182)
groupB = np.array([1] * 27 + [0] * 173 )

In [96]:
def diff(permA, permB):
    return np.mean(permB) - np.mean(permA)
def perm_reps(groupA, groupB):
    data = np.concatenate((groupA,groupB), axis=0)
    perm_data = np.random.permutation(data)
    return perm_data[:len(groupA)] , perm_data[len(groupA):]

def AB_testing(groupA, groupB, perm_reps, n_draws):
    perm_rep = np.empty(n_draws)
    for i in range(n_draws):
        permA, permB = perm_reps(groupA, groupB)
        perm_rep[i] = diff(permA, permB)
    return 'p_value is {}'.format(np.sum(perm_rep >= diff(groupA, groupB))/float(n_draws))

In [97]:
AB_testing(groupA, groupB, perm_reps, 10000)

'p_value is 0.1005'

### Boostrap hypothesis test for post compaign analysis (binary)

In [86]:
control = np.array( [1] * 18 + [0] * 182)
treat = np.array([1] * 135 + [0] * 865 )

In [102]:
diff(control,treat)

0.0450

In [111]:
def bs_reps(control,treat):
    return np.random.choice(control, size=len(control), replace=True), np.random.choice(treat, size=len(treat), replace=True)

In [112]:
a, b = bs_reps(control, treat)

In [120]:
def post_compn(control, treat, bs_reps, diff, n_draws):
    bs_rep = np.empty(n_draws)
    for i in range(n_draws):
        bsC, bsT = bs_reps(control,treat)
        bs_rep[i] = diff(bsC, bsT)
    return 'p-value is {}'.format(np.sum(bs_rep >= diff(control, treat))/float(n_draws))
    
    

In [121]:
post_compn(control, treat, bs_reps, diff, 10000)

'p-value is 0.502'

### Boostrap hypothesis test for post compaign analysis (continue number)

In [129]:
control = np.random.rand(200) * 100
treat = np.random.rand(1000) * 100

In [132]:
diff(control,treat)

3.7297

In [138]:
def data_shifted_mean(control, treat):
    mean_concat = np.mean(np.concatenate((control,treat), axis=0))
    C_shifted = control - np.mean(control) + mean_concat
    T_shifted = treat - np.mean(treat) + mean_concat
    return C_shifted, T_shifted

In [149]:
def bs_continue(control, treat, diff, data_shifted_mean, n_draws):
    bs_cont = np.empty(n_draws)
    C_shifted, T_shifted = data_shifted_mean(control, treat)
    for i in range(n_draws):
        C_shift_bs, T_shift_bs = bs_reps(C_shifted,T_shifted)
        bs_cont[i] = diff(C_shift_bs, T_shift_bs)
    return 'p-value is {}'.format(np.sum(bs_cont >= diff(control,treat))/float(n_draws))    
        

In [150]:
bs_continue(control,treat,diff, data_shifted_mean, 10000)

'p-value is 0.0455'